In [3]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn import *
import math as math 
from numpy.linalg import norm
%matplotlib inline
import matplotlib.pyplot as plt
import datetime

C:\Program Files (x86)\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Program Files (x86)\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
C:\Program Files (x86)\Anaconda3\lib\site-packages\sklearn\lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarnin

In [4]:
#Train

train = pd.read_csv('train.csv', delimiter=',', index_col='datetime', parse_dates=True)
#train.describe()

train['year'] = train.index.year
train['month'] = train.index.month
train['hour'] = train.index.hour
train['day'] = train.index.weekday

season = train['month']
season = season.apply(lambda s: 4 if int(s) == 1 or int(s) == 2 else (int(s) - 3) // 3 + 1)   
train = train.drop('season', axis = 1)
train.insert(5, 'season', season)

hour = train['hour']
mainHour = np.arange(hour.shape[0])
mainHour = [1 if (hour[i] == 7) or (hour[i] == 8) or (hour[i] == 17) or (hour[i] == 18) else 0 for i in range(hour.shape[0])]
train.insert(6, 'mainHour', mainHour)

train.head(5)

,holiday,workingday,weather,temp,atemp,season,mainHour,humidity,windspeed,casual,registered,count,year,month,hour,day
datetime,,,,,,,,,,,,,,,,
2011-01-01 00:00:00,0,0,1,9.84,14.395,4,0,81,0.0,3,13,16,2011,1,0,5
2011-01-01 01:00:00,0,0,1,9.02,13.635,4,0,80,0.0,8,32,40,2011,1,1,5
2011-01-01 02:00:00,0,0,1,9.02,13.635,4,0,80,0.0,5,27,32,2011,1,2,5
2011-01-01 03:00:00,0,0,1,9.84,14.395,4,0,75,0.0,3,10,13,2011,1,3,5
2011-01-01 04:00:00,0,0,1,9.84,14.395,4,0,75,0.0,0,1,1,2011,1,4,5


In [5]:
#Test
 
test = pd.read_csv('test.csv', delimiter=',', index_col='datetime', parse_dates=True)
#test.describe()

test['year'] = test.index.year
test['month'] = test.index.month
test['hour'] = test.index.hour
test['day'] = test.index.weekday

season = test['month']
season = season.apply(lambda s: 4 if int(s) == 1 or int(s) == 2 else (int(s) - 3) // 3 + 1)   
test = test.drop('season', axis = 1)
test.insert(5, 'season', season)

hour = test['hour']
mainHour = np.arange(hour.shape[0])
mainHour = [1 if (hour[i] == 7) or (hour[i] == 8) or (hour[i] == 17) or (hour[i] == 18) else 0 for i in range(hour.shape[0])]
test.insert(6, 'mainHour', mainHour)

test.head(5)

,holiday,workingday,weather,temp,atemp,season,mainHour,humidity,windspeed,year,month,hour,day
datetime,,,,,,,,,,,,,
2011-01-20 00:00:00,0,1,1,10.66,11.365,4,0,56,26.0027,2011,1,0,3
2011-01-20 01:00:00,0,1,1,10.66,13.635,4,0,56,0.0000,2011,1,1,3
2011-01-20 02:00:00,0,1,1,10.66,13.635,4,0,56,0.0000,2011,1,2,3
2011-01-20 03:00:00,0,1,1,10.66,12.880,4,0,56,11.0014,2011,1,3,3
2011-01-20 04:00:00,0,1,1,10.66,12.880,4,0,56,11.0014,2011,1,4,3


In [23]:
learn_rate = 0.1
maxDepth = 6
nEstimators = 90
subSample = 0.85
obj='reg:linear'
regAlpha = 0
regLambda = 0.9

train_first_year = train.ix[0:5422,]
train_second_year = train.ix[5422:train.shape[0],]


X = train_first_year.drop(['mainHour', 'casual', 'registered', 'count'], axis = 1)
Xr = train_first_year.drop(['casual', 'registered', 'count'], axis = 1)
X_test = train_second_year.drop(['mainHour', 'casual', 'registered', 'count'], axis = 1)
Xr_test = train_second_year.drop(['casual', 'registered', 'count'], axis = 1)

y = np.log(train_first_year['casual'] + 1)
xgb_model = xgb.XGBRegressor(learning_rate = learn_rate, max_depth = maxDepth, n_estimators = nEstimators,
                             objective = obj, subsample = subSample, reg_alpha = regAlpha, reg_lambda = regLambda).fit(X,y)

#xgb_model = xgb.XGBRegressor(learning_rate = learn_rate, max_depth = maxDepth, n_estimators = nEstimators, subsample = subSample).fit(X,y)

predictions1 = np.exp(xgb_model.predict(X_test)) - 1
#-----------------------------------
y = np.log(train_first_year['registered'] + 1)
xgb_model = xgb.XGBRegressor(learning_rate = learn_rate, max_depth = maxDepth, n_estimators = nEstimators,  
                             objective = obj, subsample = subSample, reg_alpha = regAlpha, reg_lambda = regLambda).fit(Xr,y)
# xgb_model = xgb.XGBRegressor(learning_rate = learn_rate, max_depth = maxDepth, n_estimators = nEstimators, subsample = subSample).fit(Xr,y)
predictions2 = np.exp(xgb_model.predict(Xr_test)) - 1
#-----------------------------------
y = train_second_year['count']
predictions = predictions1 + predictions2
predictions = np.round(predictions)
predictions = [0 if predictions[i] < 0.01 else predictions[i] for i in range(predictions.shape[0])]
predictions = np.asarray(predictions)

print (math.sqrt(1 / len(y) * sum((np.log(predictions + 1) - np.log(y + 1))**2)))

0.5833504396465444


In [ ]:
 #xgb.plot_importance(xgb_model)